In [115]:
!pip install jq

In [116]:
!pip install langchain langchain-openai langchain-pinecone langchain-community

In [117]:
!pip install --upgrade langchain

In [118]:
from google.colab import userdata
from langchain.document_loaders import JSONLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone, os, json
from langchain.schema import Document
# from langchain.document_loaders import JSONLinesLoader

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY_2')

In [119]:
os.environ['OPENAI_EMBEDDING_MODEL'] = userdata.get('OPENAI_EMBEDDING_MODEL')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [155]:
loader = JSONLoader(
    file_path="output_cleaned.jsonl",
    jq_schema=".",               # 한 줄 전체 Object
    text_content=False,# 본문 필드
    json_lines=True,        # ← jsonl 형식임을 알려줌

)
documents = loader.load()
documents

[Document(metadata={'source': '/content/output_cleaned.jsonl', 'seq_num': 1}, page_content='{"ENTRPS": "\\uc77c\\ub3d9\\ubc14\\uc774\\uc624\\uc0ac\\uc774\\uc5b8\\uc2a4(\\uc8fc)", "PRDUCT": " 11\\uc885 \\ud63c\\ud569\\uc720\\uc0b0\\uade0", "DISTB_PD": "\\uc81c\\uc870\\uc77c\\ub85c\\ubd80\\ud130 24\\uac1c\\uc6d4\\uae4c\\uc9c0", "SRV_USE": "\\uac74\\uac15\\uae30\\ub2a5\\uc2dd\\ud488 \\uc6d0\\ub8cc\\ub85c \\uc0ac\\uc6a9", "PRSRV_PD": "\\ub0c9\\uc7a5\\uc870\\uac74", "INTAKE_HINT1": "1. \\uc9c8\\ud658\\uc774 \\uc788\\uac70\\ub098 \\uc758\\uc57d\\ud488 \\ubcf5\\uc6a9 \\uc2dc \\uc804\\ubb38\\uac00\\uc640 \\uc0c1\\ub2f4\\ud558\\uc2ed\\uc2dc\\uc624.\\n2. \\uc54c\\ub808\\ub974\\uae30 \\uccb4\\uc9c8 \\ub4f1\\uc740 \\uac1c\\uc778\\uc5d0 \\ub530\\ub77c \\uacfc\\ubbfc\\ubc18\\uc751\\uc744 \\ub098\\ud0c0\\ub0bc \\uc218 \\uc788\\uc2b5\\ub2c8\\ub2e4.\\n3. \\uc5b4\\ub9b0\\uc774\\uac00 \\ud568\\ubd80\\ub85c \\uc12d\\ucde8\\ud558\\uc9c0 \\uc54a\\ub3c4\\ub85d \\uc77c\\uc77c\\uc12d\\ucde8\\ub7c9 \\ubc29\\ubc

In [121]:
from pprint import pprint

first = documents[0]
print("page_content 타입:", type(first.page_content))
pprint(first.page_content)
print("metadata 타입:", type(first.metadata))
pprint(first.metadata)

page_content 타입: <class 'str'>
('{"ENTRPS": '
 '"\\uc77c\\ub3d9\\ubc14\\uc774\\uc624\\uc0ac\\uc774\\uc5b8\\uc2a4(\\uc8fc)", '
 '"PRDUCT": " 11\\uc885 \\ud63c\\ud569\\uc720\\uc0b0\\uade0", "DISTB_PD": '
 '"\\uc81c\\uc870\\uc77c\\ub85c\\ubd80\\ud130 24\\uac1c\\uc6d4\\uae4c\\uc9c0", '
 '"SRV_USE": "\\uac74\\uac15\\uae30\\ub2a5\\uc2dd\\ud488 \\uc6d0\\ub8cc\\ub85c '
 '\\uc0ac\\uc6a9", "PRSRV_PD": "\\ub0c9\\uc7a5\\uc870\\uac74", "INTAKE_HINT1": '
 '"1. \\uc9c8\\ud658\\uc774 \\uc788\\uac70\\ub098 \\uc758\\uc57d\\ud488 '
 '\\ubcf5\\uc6a9 \\uc2dc \\uc804\\ubb38\\uac00\\uc640 '
 '\\uc0c1\\ub2f4\\ud558\\uc2ed\\uc2dc\\uc624.\\n2. '
 '\\uc54c\\ub808\\ub974\\uae30 \\uccb4\\uc9c8 \\ub4f1\\uc740 '
 '\\uac1c\\uc778\\uc5d0 \\ub530\\ub77c \\uacfc\\ubbfc\\ubc18\\uc751\\uc744 '
 '\\ub098\\ud0c0\\ub0bc \\uc218 \\uc788\\uc2b5\\ub2c8\\ub2e4.\\n3. '
 '\\uc5b4\\ub9b0\\uc774\\uac00 \\ud568\\ubd80\\ub85c '
 '\\uc12d\\ucde8\\ud558\\uc9c0 \\uc54a\\ub3c4\\ub85d '
 '\\uc77c\\uc77c\\uc12d\\ucde8\\ub7c9 \\ubc29\\ubc95\

In [122]:
new_docs = []
with open("output_cleaned.jsonl", "r", encoding="utf-8") as f:
    for raw in f:
        rec = json.loads(raw)
        # ▶ 모든 필드를 하나의 텍스트로 합칩니다.
        #   "키: 값" 형식으로 줄바꿈을 넣어 주면 가독성도 좋습니다.
        parts = []
        for k, v in rec.items():
            # None 은 빈 문자열, 숫자나 불리언은 str() 처리
            val = "" if v is None else str(v)
            parts.append(f"{k}: {val}")
        combined_text = "\n".join(parts)

        # ▶ page_content 에 합친 텍스트, metadata 에는 원본 dict
        new_docs.append(Document(page_content=combined_text, metadata=rec))

In [123]:
for i, d in enumerate(new_docs[:5], 1):
    print(f"문서 {i} 본문:\n{d.page_content}\n")
    print(f"문서 {i} 메타데이터 키들: {list(d.metadata.keys())}\n")
    print("-"*40)

문서 1 본문:
ENTRPS: 일동바이오사이언스(주)
PRDUCT:  11종 혼합유산균
DISTB_PD: 제조일로부터 24개월까지
SRV_USE: 건강기능식품 원료로 사용
PRSRV_PD: 냉장조건
INTAKE_HINT1: 1. 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.
2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.
3. 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도해 주십시오.
4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
MAIN_FNCTN: 유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음
BASE_STANDARD: 1. 성상 : 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 있는 분말
2. 프로바이오틱스 수(표시량 300,000,000,000(3,000억) CFU/g) : 표시량 이상
3. 대장균군 : 음성
4. 납 : 1.0 mg/kg 이하
5. 카드뮴 : 0.3 mg/kg 이하

문서 1 메타데이터 키들: ['ENTRPS', 'PRDUCT', 'DISTB_PD', 'SRV_USE', 'PRSRV_PD', 'INTAKE_HINT1', 'MAIN_FNCTN', 'BASE_STANDARD']

----------------------------------------
문서 2 본문:
ENTRPS: 주식회사 성윤 에프엔지(F&G)
PRDUCT:  6년근 고려홍삼정 PREMIUM
DISTB_PD: 제조일로부터 24개월
SRV_USE: 1일 3회, 1회 1스푼(3.2g)씩  물에 녹여 섭취 하십시요.

PRSRV_PD: 직사광선을 피해 건조하고 서늘한 곳에서 보관한다.

INTAKE_HINT1: [홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의
2) 특이체질등 알레르기 체질의 경우 제품성분을 확인 후 섭취하시기 바랍니다.
 3) 15세 이하의 어린이는 상기 섭취량의 절반 정도를 섭취하시요.
 4) 제품 개봉 또는 섭취시에 포장재로 인한 상처를 입을수 있으니주의

In [124]:
null_entries = {}

for idx, doc in enumerate(new_docs):
    # metadata 중 값이 None인 키만 뽑아내기
    null_keys = [k for k, v in doc.metadata.items() if v is None]
    if null_keys:
        null_entries[idx] = null_keys

# 결과 출력
if null_entries:
    print("👀 metadata에 None(NULL) 값이 발견된 문서들:")
    for idx, keys in null_entries.items():
        print(f"  - 문서 {idx} 에서 null인 키: {keys}")
else:
    print("✔️ 모든 metadata 값이 None(NULL)이 아닙니다.")

✔️ 모든 metadata 값이 None(NULL)이 아닙니다.


In [125]:
# 문서로부터 벡터스토어 생성
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


# 임베딩 모델
embeddings = OpenAIEmbeddings(model=os.environ['OPENAI_EMBEDDING_MODEL'])

vector_store = PineconeVectorStore.from_documents(
    new_docs,
    embedding=embeddings,
    index_name='project'
)


In [126]:
# 존재하는 인덱스에 접근/검색
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
vector_store = PineconeVectorStore(
    index=pc.Index('project'),
    embedding=embeddings
)

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3}
)


In [127]:
docs = retriever.invoke("비타민 제품 알려줘")
for i in docs:
    pprint(i)

Document(id='5b11e062-5dd4-47d4-9e57-99203332088c', metadata={'BASE_STANDARD': '(1) 성상 : 고유의 향미가 있고 이미, 이취가 없는 흑갈색 또는 어두운 회갈색의 장방형 제피정제\n(2) 나이아신 : 표시량(150 mgNE/1.2g)의 80%~150%\n(3) 판토텐산 : 표시량(50 mg/1.2g)의 80%~180%\n(4) 비타민B1 : 표시량(12 mg/1.2g)의 80%~180%\n(5) 비타민B2 : 표시량(14 mg/1.2g)의 80%~180%\n(6) 비타민B12 : 표시량(24 ㎍/1.2g)의 80%~180%\n(7) 비오틴 : 표시량(300 ㎍/1.2g)의 80%~180%    \n(8) 대장균군 : 음성\n(9) 붕해 : 적합(60분 이내)', 'DISTB_PD': '제조일로부터 24개월', 'ENTRPS': '(주)서흥헬스케어 2공장', 'INTAKE_HINT1': '(1) 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n(2) 특정 성분에 알레르기가 있으신 분은 원료명을 확인 후 섭취하십시오.', 'MAIN_FNCTN': '나이아신: ①체내 에너지 생성에 필요\n\n판토텐산: ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요\n\n비타민B1: ①탄수화물과 에너지 대사에 필요\n\n비타민B2: ①체내 에너지 생성에 필요\n\n비타민B12: ①정상적인 엽산 대사에 필요\n\n비오틴: ①지방, 탄수화물, 단백질 대사와 에너지 생성에 필요', 'PRDUCT': ' 비타민B 컴플렉스 ', 'PRSRV_PD': '(1) 제품은 직사광선을 피하고 실온에서 보관 유통하십시오.   (2) 제품 개봉 후에는 흡습되지 않게 밀봉하여 보관하십시오.  ', 'SRV_USE': '1일 1회, 1회 2정(1.2g)을 충분한 물과 함께 섭취하십시오.'}, page_content='ENTRPS: (주)서흥헬스케어 2공장\nPRDUCT:  비타민B 컴플렉스 \nDISTB_PD: 제조일로부터 2

In [128]:
docs = retriever.invoke("6년근 홍삼 자신만만 홍삼스틱 하루에 몇개 먹어?")
pprint(docs)

[Document(id='10a92c0f-116f-4ae4-beba-13eed1cfc4de', metadata={'BASE_STANDARD': '⑴ 성상: 고유의 색택과 향미를 가지며 이미와 이취가 없는 액상\n⑵ 진세노사이드 Rg1, Rb1 및 Rg3의 합: 표시량(11.0 mg/10 g)의 80% 이상\n⑶ 세균수(CFUs/g, 1mL당): 100  이하\n⑷ 대장균군: 음성', 'DISTB_PD': '제조일부터      2  년', 'ENTRPS': '(주)일화', 'INTAKE_HINT1': '⑴ 특수체질이거나 알레르기 체질인 경우에는 섭취 전에 반드시 원료명 및 함량을 확인하시기 바랍니다.\n⑵ 제품 개봉 또는 섭취 시 포장재에 의한 상처에 주의하시기 바랍니다.\n⑶ 의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의 하십시오.\n⑷ 유통기한이 경과된 제품은 섭취하지 마십시오.\n', 'MAIN_FNCTN': '  ⑴ 면역력 증진에 도움을 줄 수 있음\n  ⑵ 피로개선에 도움을 줄 수 있음\n  ⑶ 혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음\n  ⑷ 기억력 개선에 도움을 줄 수 있음\n  ⑸ 항산화에 도움을 줄 수 있음', 'PRDUCT': ' 6년근 홍삼 자신만만 홍삼스틱', 'PRSRV_PD': ' 습기와 직사광선을 피하여 실온에 보관하시고, 개봉 후에는 반드시 냉장 보관하십시오.', 'SRV_USE': '1일 1회, 1회 1포를 섭취하십시오. '}, page_content='ENTRPS: (주)일화\nPRDUCT:  6년근 홍삼 자신만만 홍삼스틱\nDISTB_PD: 제조일부터      2  년\nSRV_USE: 1일 1회, 1회 1포를 섭취하십시오. \nPRSRV_PD:  습기와 직사광선을 피하여 실온에 보관하시고, 개봉 후에는 반드시 냉장 보관하십시오.\nINTAKE_HINT1: ⑴ 특수체질이거나 알레르기 체질인 경우에는 섭취 전에 반드시 원료명 및 함량을 확인하시기 바랍니다.\n⑵ 제품 개봉 또는 섭취 시 포장재에 의한 상처에

In [129]:
docs = retriever.invoke("아이들이 먹을 수 있는 비타민 있어?")
pprint(docs)

[Document(id='4c3e08e2-f32e-4bf6-883d-7e3eb8e92212', metadata={'BASE_STANDARD': '1) 성상 : 고유의 향미가 있고 이미, 이취가 없는  노란색의 젤리\n2) 비타민B6 : 표시량(1.5mg/15g)의 80~150%\n3) 비타민D : 표시량(10ug/15g)의 80~180%\n4) 비타민E : 표시량(11mg α-TE/15g)의 80~150%\n5) 아연 : 표시량(8.5mg/15g) 80~150%\n6) 대장균군 : 음성', 'DISTB_PD': '제조일로부터 18개월. ', 'ENTRPS': '코스맥스바이오(주)', 'INTAKE_HINT1': '섭취 시 위장장애, 소화불량의 증상이 있을 경우 섭취를 중단하십시오. \n개인의 신체 상태에 따라 이상 증상이 생길 경우 섭취를 중단하십시오.  \n섭취 전 제품에 이상이 있는 경우 섭취를 금하십시오. \n특정 원료 성분에 알레르기 체질은 원료 성분을 확인 후 섭취 하십시오. \n제품 개봉 시 포장재에 의해 상처를 입을 위험이 있고, 개봉 시 내용물이 흘러나올 수 있으므로취급 시 주의하십시오. \n고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하십시오\n이상사례 발생시 섭취를 중단하고 전문가와 상담하십시오.    ', 'MAIN_FNCTN': '1) 비타민B6 : 단백질 및 아미노산 이용에 필요, 혈액의 호모시스테인 수준을 정상으로 유지하는데 필요.\n2) 비타민D : 칼슘과 인이 흡수되고 이용되는데 필요, 뼈의 형성과 유지에 필요, 골다공증발생 위험 감소에 도움을 줌. \n3) 비타민E : 항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요.\n4) 아연 : 정상적인 면역기능에 필요, 정상적인 세포분열에 필요. ', 'PRDUCT': ' 데일리 퍼플 키즈 멀티비타젤리 ', 'PRSRV_PD': '수분 및 열에 의해 영향을 받을 수 있으므로 직사광선을 피해 서늘한 곳에 보관하십시오. 어린이 손에 닿지 않는 곳에 보관하십시오.', 'SRV_USE

In [150]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import RegexParser

prompt = PromptTemplate.from_template("""
[System Instruction]
- 당신은 여러문서를 분석하여 사용자의 질문에 친절히 답변하는 건강기능식품 전문가이다.
- 만약, 증상을 입력하면 증상에 대한 공감을 먼저 한 뒤, 제품을 추천해줘야 한다.
- 추천한 제품은 자세하게 설명해줘야 한다.
- 제품을 추천할 시, 3개의 제품을 추천해야 한다. 만약, 3개의 제품이 없다면, 있는 만크만 추천해준다
- 답변은 반드시 [Example - Output Indicator]에 따라야 한다.
- 주어진 문서내에서만 정보를 추출해 답변해야 한다.
- 사용자의 질문에 대한 내용을 주어진 문서상에서 찾을 수 없는 경우 찾을수 없다고 답변해야 한다.
- 절대 말을 지어내어서는 안된다!!!
- ~것으로 끝나지 않도록 문장으로 답변해야 한다.

[Context]
{context}

[Input Data]
{question}

[Example - Output Indicator]
Q: 11종 혼합유산균의 유통기한은?
A: 제조일로부터 24개월입니다.

Q: 11종 혼합유산균의 섭취시 주의사항은?
A: 1. 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.
   2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.
   3. 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도해 주십시오.
   4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.

""")
llm = ChatOpenAI(model='gpt-4o-mini', temperature=.3)

# re.DOTALL: .이 \n가 포함해 검색하는 flag
# - inline flag버젼: (?s)
regex_parser = RegexParser(regex=r'(?s)(.*)', output_keys=['answer'])

chain = prompt | llm | regex_parser

query = '혈당마스터 (Diabet master)(수출용) 섭취시 주의사항은?'
retrieved = retriever.invoke(query)
response = chain.invoke({
    'context': '\n\n'.join(doc.page_content for doc in retrieved) ,
    'question': query
})
# print(response)
print(response['answer'])

혈당마스터 (Diabet master)(수출용) 섭취 시 주의사항은 다음과 같습니다. 

1. 과량섭취 시 부작용이 있을 수 있습니다.
2. 특정 원료에 알레르기가 있거나 질병 치료 약물 투여 중인 분은 섭취 전 전문가와 상담해야 합니다.
3. 소비기한이 경과된 제품은 섭취하지 않아야 합니다.
4. 당뇨병이 있는 경우 섭취 전 전문가와 상담할 것을 권장합니다.
5. 이상 사례 발생 시 섭취를 중단하고 전문가와 상담해야 합니다.


In [138]:
query = '비오틴 효모 더 채움은 어떻게 섭취해야해?'
retrieved = retriever.invoke(query)
response = chain.invoke({
    'context': '\n\n'.join(doc.page_content for doc in retrieved) ,
    'question': query
})
# print(response)
print(response['answer'])

비오틴 효모 더 채움은 1일 3회, 1회 10정을 물과 함께 섭취해야 합니다.


In [151]:
query = '유산균 제품 추천해줘'
retrieved = retriever.invoke(query)
response = chain.invoke({
    'context': '\n\n'.join(doc.page_content for doc in retrieved) ,
    'question': query
})
# print(response)
pprint(response['answer'])

('유산균 제품을 찾고 계신 것에 대해 공감합니다. 유산균은 장 건강에 매우 중요한 역할을 하며, 많은 분들이 이를 통해 건강을 개선하고자 '
 '합니다. 아래에서 추천드리는 유산균 제품들을 확인해 보세요.\n'
 '\n'
 '1. **중원프로바이오틱스 유산균**\n'
 '   - **주요 기능**: 유산균 증식 및 유해균 억제에 도움을 줄 수 있으며, 배변활동을 원활하게 하는 데 도움을 줄 수 있습니다.\n'
 '   - **섭취 방법**: 1일 1회, 1포(2g)를 섭취합니다.\n'
 '   - **성상**: 고유의 향미를 가지는 회백색의 분말로, 프로바이오틱스 수는 300,000,000CFU/2,000mg입니다.\n'
 '   - **보관 방법**: 실온에서 18개월 동안 보관할 수 있습니다.\n'
 '\n'
 '2. **낙산균(Naksan)(수출용)**\n'
 '   - **주요 기능**: 유산균 증식 및 유해균 억제, 배변활동 원활, 장 건강에 도움을 줄 수 있습니다.\n'
 '   - **섭취 방법**: 1일 1회, 1포를 물 혹은 직접 섭취합니다.\n'
 '   - **성상**: 고유의 향미가 있으며, 검정색과 갈색 알갱이가 있는 밝은 회황색 분말입니다. 프로바이오틱스 수는 '
 '10,000,000CFU/2,000mg 이상입니다.\n'
 '   - **보관 방법**: 고온다습한 곳과 직사광선을 피하여 습기가 적고 건조한 곳에 보관해야 합니다.\n'
 '\n'
 '이 두 가지 제품은 유산균의 효과를 잘 나타내며, 사용하기도 간편합니다. 건강한 장을 위해 적절한 유산균 섭취를 고려해 보시기 바랍니다.')


In [152]:
query = input()
retrieved = retriever.invoke(query)
response = chain.invoke({
    'context': '\n\n'.join(doc.page_content for doc in retrieved) ,
    'question': query
})
# print(response)
pprint(response['answer'])

쓰리플렉스 콜레스테롤∙ 혈당 ∙혈압은 임산부가 먹어도 되나요?
('쓰리플렉스 콜레스테롤∙ 혈당 ∙혈압은 임산부가 섭취해서는 안 됩니다. 이 제품은 어린이, 임산부 및 수유부는 섭취를 피해야 한다고 '
 '명시되어 있습니다. 건강을 위해서는 항상 전문가와 상담 후에 섭취 결정을 하시는 것이 좋습니다.')


In [154]:
query = input()
retrieved = retriever.invoke(query)
response = chain.invoke({
    'context': '\n\n'.join(doc.page_content for doc in retrieved) ,
    'question': query
})
# print(response)
pprint(response['answer'])

젤리 형태로 되어 있는 비타민 추천해줘
('젤리 형태의 비타민을 찾고 계시군요. 젤리 형태의 비타민은 어린이들이나 젤리의 맛을 선호하는 분들에게 좋은 선택입니다. 아래에 추천할 '
 '제품을 소개해 드리겠습니다.\n'
 '\n'
 '1. **데일리 퍼플 키즈 멀티비타젤리**\n'
 '   - **주요 성분 및 기능**:\n'
 '     - 비타민B6: 단백질 및 아미노산 이용에 필요하며, 혈액의 호모시스테인 수준을 정상으로 유지하는 데 도움을 줍니다.\n'
 '     - 비타민D: 칼슘과 인의 흡수 및 이용에 필요하며, 뼈의 형성과 유지에 도움을 줍니다. 골다공증 발생 위험 감소에도 '
 '기여합니다.\n'
 '     - 비타민E: 항산화 작용을 통해 유해산소로부터 세포를 보호하는 데 필요합니다.\n'
 '     - 아연: 정상적인 면역 기능과 세포 분열에 필요합니다.\n'
 '   - **섭취 방법**: 1일 1회, 1회 1포(15g)를 씹어서 섭취합니다.\n'
 '   - **보관 방법**: 수분 및 열에 영향을 받을 수 있으므로 직사광선을 피해 서늘한 곳에 보관하고, 어린이 손에 닿지 않는 곳에 '
 '보관해야 합니다.\n'
 '\n'
 '이 제품은 어린이와 성인 모두에게 적합하며, 맛있게 섭취할 수 있는 젤리 형태로 되어 있어 건강 관리에 도움을 줄 수 있습니다. '
 '추가적으로 다른 젤리 형태의 비타민 제품은 현재로서는 이 제품 외에 정보가 없습니다. 필요하신 경우, 다른 형태의 비타민도 고려해 보시는 '
 '것이 좋습니다.')
